用于创建多组aadt流量图
方法一使用folium库直接获取地图
方法二使用geopanda生成加州地图

In [ ]:
import folium
import branca.colormap as cm
import numpy as np
import pandas as pd
import time
from IPython.display import display, clear_output

def generate_aadt_map(df, num_subsegments=10):
    """
    Generate AADT traffic volume map based on DataFrame, with color gradient according to AADT size
    """

    df["Longitude"] = df["Longitude"].apply(lambda x: -x if x > 0 else x)# Correct longitude direction

    # Create a Folium map
    m = folium.Map(
        location=[df["Latitude"].mean(), df["Longitude"].mean()], 
        zoom_start=7, 
        tiles="OpenStreetMap"
    )

    # Color mapping
    colormap = cm.LinearColormap(
        colors=["blue", "green", "yellow", "red"],  
        vmin=df["VEHICLE_AADT_TOTAL"].min(),  
        vmax=df["VEHICLE_AADT_TOTAL"].max()
    )

    ## Draw smooth gradient lines
    for i in range(len(df) - 1):
        lat1, lon1, aadt1 = df.iloc[i][["Latitude", "Longitude", "VEHICLE_AADT_TOTAL"]]
        lat2, lon2, aadt2 = df.iloc[i + 1][["Latitude", "Longitude", "VEHICLE_AADT_TOTAL"]]

        # Generate interpolation (subdivide line segments)
        lats = np.linspace(lat1, lat2, num_subsegments)
        lons = np.linspace(lon1, lon2, num_subsegments)
        aadts = np.linspace(aadt1, aadt2, num_subsegments)  

        # Draw smaller line segments with color gradient
        for j in range(num_subsegments - 1):
            color = colormap(aadts[j])  

            folium.PolyLine(
                locations=[(lats[j], lons[j]), (lats[j + 1], lons[j + 1])],
                color=color,
                weight=5,
                opacity=0.8
            ).add_to(m)

    # Add AADT color legend
    colormap.caption = "AADT traffic data"
    colormap.add_to(m)

    return m

def animate_aadt_maps(csv_files, delay=3):
    """
    Display AADT maps by year, creating a simple animation effect
    
    Parameters:
    - csv_files (dict): A dictionary containing year and CSV file path, e.g., 
        {2019: "data_2019.csv", 2020: "data_2020.csv", ...}
    - delay (int): Time (in seconds) to display each year's map
    """
    for year, file in sorted(csv_files.items()):
        # Read data
        df = pd.read_csv(file)
        m = generate_aadt_map(df)

        # Clear previous map and display the new map
        clear_output(wait=True)
        print(f"显示 {year} 年的交通流量地图")
        display(m)

        time.sleep(delay)

# Example: Define data files for multiple years
csv_files = {
    2019: "i5_2019_coordinates.csv",
    2020: "i5_2020_coordinates.csv",
    2021: "i5_2021_coordinates.csv",
    2022: "i5_2022_coordinates.csv"
}

# Run the animation (each year displayed for 3 seconds)
animate_aadt_maps(csv_files, delay=3)


In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import imageio
import os

def load_california_map():
    """
    Load California map data:
    1. Download Natural Earth world map and filter for California
    2. Or use a local Shapefile (manual download required)
    """
    try:
        # 1 Try loading Natural Earth data (if already downloaded)
        world = gpd.read_file("https://naturalearth.s3.amazonaws.com/10m_cultural/ne_10m_admin_1_states_provinces.zip")
        california = world[(world["admin"] == "United States of America") & (world["name"] == "California")]
        return california
    except:
        # 2 Fallback: Use local Shapefile
        shapefile_path = "california_shapefile.shp"   # You need to provide the California boundary file
        california = gpd.read_file(shapefile_path)
        return california

def load_aadt_data(csv_file):
    """
    Load AADT CSV data and convert it to a GeoDataFrame.
    """
    df = pd.read_csv(csv_file)
    df["Longitude"] = df["Longitude"].apply(lambda x: -x if x > 0 else x)  # 修正经度
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
    return gdf

def plot_aadt_map(california, gdf, year, ax, colormap, norm):
    """
    Plot California map and overlay AADT lines on the Matplotlib axis.
    """
    # Plot the California map

    california.plot(ax=ax, color="whitesmoke", edgecolor="black")

    # Overlay AADT lines
    for i in range(len(gdf) - 1):
        lat1, lon1, aadt1 = gdf.iloc[i][["Latitude", "Longitude", "VEHICLE_AADT_TOTAL"]]
        lat2, lon2, aadt2 = gdf.iloc[i + 1][["Latitude", "Longitude", "VEHICLE_AADT_TOTAL"]]

        # Choose color based on AADT
        color = colormap(norm((aadt1 + aadt2) / 2))

        # Plot the lines
        ax.plot([lon1, lon2], [lat1, lat2], color=color, linewidth=2)

    ax.set_title(f"AADT traffic volume - {year}", fontsize=14)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")

def create_aadt_animation(csv_files, output_gif="aadt_animation.gif", delay=1.5):
    """
    Generate GIF animation of AADT traffic volume changes over different years (overlayed on California map).
    """
    images = []  # Store image paths
    california = load_california_map()  # Load California map

    # Color mapping
    cmap = plt.get_cmap("jet")  # Color gradient
    all_aadt_values = []  # Store all AADT values

    # First, iterate over files to gather AADT statistics
    for file in csv_files.values():
        gdf = load_aadt_data(file)
        all_aadt_values.extend(gdf["VEHICLE_AADT_TOTAL"])

    norm = plt.Normalize(min(all_aadt_values), max(all_aadt_values))  # Normalize AADT values


    for year, file in sorted(csv_files.items()):
        gdf = load_aadt_data(file)

        # create the canvas
        fig, ax = plt.subplots(figsize=(8, 6))
        plot_aadt_map(california, gdf, year, ax, cmap, norm)

        # add colorbar
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        plt.colorbar(sm, ax=ax, label="AADT traffic volume")

        # save temporary image and get its path
        img_filename = f"aadt_{year}.png"
        plt.savefig(img_filename, dpi=100)
        plt.close()
        images.append(img_filename)  # save file path

        print(f"Generated map for {year} ")

    # create GIF 
    imageio.mimsave(output_gif, [imageio.imread(img) for img in images], duration=delay)
    print(f"GIF saved as {output_gif}")

    # clean up temporary images
    for img in images:
        os.remove(img)  # delete the temporary image files

# input csv
csv_files = {
    2019: "i5_2019_coordinates.csv",
    2020: "i5_2020_coordinates.csv",
    2021: "i5_2021_coordinates.csv",
    2022: "i5_2022_coordinates.csv"
}

# Generate AADT animation GIF
create_aadt_animation(csv_files, output_gif="aadt_california_traffic.gif", delay=5)
